In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [6]:
from google.colab import drive
drive.mount('/content/drive')
train_path = "/content/drive/MyDrive/VIR_AJL_Team_Glycerin/Dermatology_Dataset/bttai-ajl-2025_unzipped/train.csv"
test_path = "/content/drive/MyDrive/VIR_AJL_Team_Glycerin/Dermatology_Dataset/bttai-ajl-2025_unzipped/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_dir = '/content/drive/MyDrive/VIR_AJL_Team_Glycerin/Dermatology_Dataset/bttai-ajl-2025_unzipped/train/train'

In [28]:

def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128), training = True):
    """
    Template function to create image generators.
    Students should complete this function to load images and labels properly.
    """
    datagen = ImageDataGenerator(rescale=1./255)

    # Create the generator
    if training:
        # For training data, include labels
        generator = datagen.flow_from_dataframe(
            dataframe=dataframe,
            directory=directory,
            x_col='file_path',  # Column with image file paths
            y_col='encoded_label',  # Column with labels
            target_size=target_size,
            batch_size=batch_size,
            class_mode='raw',  # Use 'raw' for numerical labels or 'categorical' for one-hot encoded labels
            validate_filenames=False,
            shuffle=True  # Shuffle data for training
        )
    else:
        # For test data, exclude labels
        generator = datagen.flow_from_dataframe(
            dataframe=dataframe,
            directory=directory,
            x_col='file_path',  # Column with image file paths
            y_col=None,  # No labels for test data
            target_size=target_size,
            batch_size=batch_size,
            class_mode=None,  # No labels
            validate_filenames=False,
            shuffle=False  # Do not shuffle test data
        )

    return generator

train_generator = create_generator(train_data, train_dir, training = True)
val_generator = create_generator(val_data, train_dir, training = False)

Found 2288 non-validated image filenames.
Found 572 non-validated image filenames.


In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model



base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


base_model.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
num_classes = 21
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
42/72 ━━━━━━━━━━━━━━━━━━━━ 18s 628ms/step - accuracy: 0.2274 - loss: 2.9690

In [14]:
model.save("/content/drive/MyDrive/VIR_AJL_Team_Glycerin/Saved_Models/my_model.keras")

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)  # Example: Rescale pixel values to [0, 1]

# Create a generator for the test images
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,  # Your test DataFrame
    directory='/content/drive/MyDrive/VIR_AJL_Team_Glycerin/Dermatology_Dataset/bttai-ajl-2025_unzipped/test/test',  # Path to the folder containing images
    x_col='md5hash',
    y_col=None,
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

Found 1227 validated image filenames.


In [24]:
class_indices = train_generator.class_indices

AttributeError: 'DataFrameIterator' object has no attribute 'class_indices'

In [23]:
test_predictions = model.predict(test_generator, steps=len(test_generator))

predicted_class_indices = np.argmax(test_predictions, axis=1)

class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

predicted_labels = [class_labels[idx] for idx in predicted_class_indices]

submission_df = pd.DataFrame({
    'md5hash': test_df['md5hash'],
    'label': predicted_labels
})

submission_df.to_csv('/content/drive/MyDrive/VIR_AJL_Team_Glycerin/submissions/submission.csv', index=False)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 256s 7s/step


AttributeError: 'DataFrameIterator' object has no attribute 'class_indices'